<a href="https://colab.research.google.com/github/prashantlal56/Deep_Learning_practice/blob/master/Copy_of_Course_3_Week_2_Lesson_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.2.0


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9PDWLB/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9PDWLB/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9PDWLB/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [7]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [9]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4911 - accuracy: 0.7466 - val_loss: 0.3361 - val_accuracy: 0.8523
Epoch 2/10
782/782 [==============================] - 7s 8ms/step - loss: 0.1030 - accuracy: 0.9722 - val_loss: 0.4375 - val_accuracy: 0.8326
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0274 - accuracy: 0.9956 - val_loss: 0.5230 - val_accuracy: 0.8272
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0059 - accuracy: 0.9996 - val_loss: 0.5785 - val_accuracy: 0.8318
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6355 - val_accuracy: 0.8312
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.6767 - val_accuracy: 0.8328
Epoch 8/10
782/782 [==============================] - 7s 9ms/step - loss: 4.8771e-04 - accuracy: 1.0000 - val_loss: 0.7160 - val_accuracy: 0.8338
Ep

In [10]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [14]:
for i in range(0,5):
  print(f"{i} word's weight in 16 dimensions : {weights[i]}.")

0 word's weight in 16 dimensions : [-0.02534925 -0.00243003  0.00857663  0.04926639  0.01589275 -0.01565299
  0.01841364  0.01877265  0.04421412 -0.02475867  0.00963593 -0.04159967
  0.05731976 -0.0058534  -0.00086352  0.02216832].
1 word's weight in 16 dimensions : [ 0.04004114  0.02180758  0.02592305  0.11884473 -0.00873121  0.02852749
 -0.02191056  0.04818282 -0.01138511 -0.02470221 -0.01029662  0.00452345
  0.04060787 -0.00972278  0.00443291 -0.00315122].
2 word's weight in 16 dimensions : [-0.02418729 -0.01466637  0.0094221   0.15574342 -0.00806354 -0.00908391
  0.01003448 -0.00264056 -0.01624747 -0.04257821  0.05194478 -0.10913763
  0.02190645 -0.01643279 -0.03921731  0.05806015].
3 word's weight in 16 dimensions : [-0.02552055 -0.07052681  0.04264836  0.08809767  0.05274297  0.02705103
 -0.02646488 -0.007877   -0.03020155 -0.03532784  0.0248136  -0.01535545
  0.04865378 -0.05205511 -0.05990998  0.05684201].
4 word's weight in 16 dimensions : [-0.02086025 -0.06357707  0.08320758 

In [16]:
[x for x in weights[0]]

[-0.025349254,
 -0.002430033,
 0.008576625,
 0.049266387,
 0.015892746,
 -0.015652988,
 0.018413639,
 0.018772647,
 0.044214122,
 -0.02475867,
 0.009635931,
 -0.04159967,
 0.057319757,
 -0.0058533982,
 -0.0008635153,
 0.02216832]

In [17]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [18]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
